In [67]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
print (urllib2.__version__)

3.5


In [89]:
#get current year
currentyear = datetime.now().year
#print(currentyear)
#generate a list of years from 1996 to current year
#the data in Securities Class Action Clearinghouse starts from 1996
yearslist = list(range(1996,currentyear,1))
#yearslist = list(range(1998,2000,1))
#print(yearslist)
#get the general link
baselink = "http://securities.stanford.edu/filings.html?filter="

In [90]:
#function to get number of pages for a year
def getPageNumber(baselink):
    pagenumberlist = []
    flag = True
    counter = 1
    while flag is True:
        link = baselink + "&page=" + str(counter)
        #print(link)
        page = urllib2.urlopen(link).read()
        #lxml parameter avoids unnecessary warning
        soup = BeautifulSoup(page,"lxml")
        pagelinks = soup.findAll("div",class_="pagination pagination-right")
        for d in pagelinks:
            lis = d.find_all('li')
        #print(len(lis))
        #print(lis[1].get_text())
        #print(lis[len(lis)-2].get_text())
        pagenumberlist.append(lis[len(lis)-2].get_text())
        #following is the logic to check number of pages:
        #the pages tab occur maximum in sets of 5 at a time and minimum only 1 page tab will be present
        #hence after the counter is incremented by 5, if the following two conditions are encountered
        #it means that there are no more pages left and hence control has shifted back to page 1
        if(counter != 1):
            if(lis[1].get_text() in ("1",lis[len(lis)-2].get_text())) and (lis[len(lis)-2].get_text() == "5" ):
                flag = False
        #print(flag) 
        counter += 5
    #print(pagenumberlist)
    #print(pagenumberlist[-2])
    return(int(pagenumberlist[-2]))

In [91]:
#This is the final code. The below cells are test codes
#Now that we have to acquire the page numbers and we can traverse through all the pages and get the filings
#information for each page
#While doing so, we will also go through each of the years from the list we defined above
#we will store the attributes from each row in separate lists 
filingName = [];filingDate = [];districtCourt = [];exchange =[];ticker = [];
for year in yearslist:
    #Find number of pages since the html has pagination feature
    link = baselink + str(year)
    #print(link)
    #invoking above function to get number of pages
    numberofpages = getPageNumber(link)
    #print(numberofpages)
    for i in range(numberofpages):
        link = baselink + str(year) + "&page=" + str(i)
        #print(link)
        page = urllib2.urlopen(link).read()
        #lxml parameter avoids unnecessary warning
        soup = BeautifulSoup(page,"lxml")
        #print(soup.prettify())
        #the tbody tag is taken instead of table tag since there is a tr attribute in thead inside table tag as well
        filingstable = soup.find('tbody')
        for row in filingstable.find_all("tr"):
            cells = row.find_all("td")
            if(len(cells)>0):
                filingName.append(cells[0].get_text().strip())
                filingDate.append(cells[1].get_text().strip())
                districtCourt.append(cells[2].get_text().strip())
                exchange.append(cells[3].get_text().strip())
                ticker.append(cells[4].get_text().strip())
            else:
                print('No data found')
                break
#store the attributes in a dataframe
filingsdata = pd.DataFrame({
        "FilingName":filingName,
        "FilingDate":filingDate,
        "DistrictCourt":districtCourt,
        "Exchange":exchange,
        "Ticker":ticker
    })
#extract the year from the Filing date column
filingsdata['FilingYear'] = filingsdata['FilingDate'].str.split('/').str[2]
#reorder the columns
cols = ['FilingName','FilingDate','FilingYear','DistrictCourt','Exchange','Ticker']
filingsdata = filingsdata[cols]
#filingsdata

In [92]:
#remove empty cells if any
filter = filingsdata["FilingName"] != ""
filingsdata = filingsdata[filter]
filingsdata.head()

,FilingName,FilingDate,FilingYear,DistrictCourt,Exchange,Ticker
20,"Access HealthNet, Inc.",12/27/1996,1996,C.D. California,NASDAQ,AHNT
21,Casino Data Systems,12/24/1996,1996,D. Nevada,NASDAQ,CSDS
22,"Dignity Partners, Inc.",12/19/1996,1996,N.D. California,NASDAQ,DPNR
23,Open Environment Corporation,12/05/1996,1996,D. Massachusetts,NASDAQ,OPEN
24,"Teletek, Inc.",12/02/1996,1996,D. Nevada,NASDAQ,TLTK


In [93]:
#write the final dataframe to a csv file
filingsdata.to_csv("C:/Users/Arko/Downloads/Studies/PersonalResearch/NetworkProject/FilingsData_SCAdata.csv")

In [ ]:
#test code(this technique works for pagination and getting correct number of pages)
#Find number of pages since the html has pagination feature
pagenumberlist = []
flag = True
counter = 1
while flag is True:
    link = baselink + str(yearslist[1]) + "&page=" + str(counter)
    print(link)
    page = urllib2.urlopen(link).read()
    #lxml parameter avoids unnecessary warning
    soup = BeautifulSoup(page,"lxml")
    pagelinks = soup.findAll("div",class_="pagination pagination-right")
    for d in pagelinks:
        lis = d.find_all('li')
    #print(len(lis))
    print(lis[1].get_text())
    print(lis[len(lis)-2].get_text())
    pagenumber.append(lis[len(lis)-2].get_text())
    if(counter != 1):
        if(lis[1].get_text() == "1") and (lis[len(lis)-2].get_text() == "5" ):
            flag = False
    print(flag) 
    counter += 5
print(pagenumberlist)
print(pagenumberlist[-2])
pagenumber = pagenumberlist[-2]

In [ ]:
#test code(this algorithm for pagination will not work)
#Find number of pages since the html has pagination feature
link = baselink + str(yearslist[1])
print(link)
page = urllib2.urlopen(link).read()
#lxml parameter avoids unnecessary warning
soup = BeautifulSoup(page,"lxml")
pages = soup.findAll("div",class_="pagination pagination-right")
for d in pages:
    lis = d.find_all('li')
print(len(lis))
#Number of pages will be total number of li tags minus the li tags next and previous tags
if(len(lis) < 2):
    numberofpages = len(lis)
else:
    numberofpages = len(lis) - 2
print(numberofpages)

In [ ]:
#test code
#the tbody tag is taken instead of table tag since there is a tr attribute in thead inside table tag as well
filingstable = soup.find('tbody')
#filingstable
#we will store the attributes from each row in separate lists 
filingName = [];filingDate = [];districtCourt = [];exchange =[];ticker = []
for row in filingstable.find_all("tr"):
    cells = row.find_all("td")
    if(len(cells)>0):
        filingName.append(cells[0].get_text().strip())
        filingDate.append(cells[1].get_text().strip())
        districtCourt.append(cells[2].get_text().strip())
        exchange.append(cells[3].get_text().strip())
        ticker.append(cells[4].get_text().strip())
    else:
        print('No data found')
        break
#print(ticker)

In [ ]:
#test code
#store the attributes in a dataframe
filingsdata = pd.DataFrame()
filingsdata = pd.DataFrame({
        "FilingName":filingName,
        "FilingDate":filingDate,
        "DistrictCourt":districtCourt,
        "Exchange":exchange,
        "Ticker":ticker
    })
#reorder the columns
cols = ['FilingName','FilingDate','DistrictCourt','Exchange','Ticker']
filingsdata = filingsdata[cols]
filingsdata